# Link Prediction

To Do:
- [x] Read: [Introduction to Graphs](https://www.analyticsvidhya.com/blog/2018/09/introduction-graph-theory-applications-python/?utm_source=blog&utm_medium=link-prediction-how-to-predict-your-future-connections-on-facebook)
- [x] Read: [Link Prediction](https://www.analyticsvidhya.com/blog/2020/01/link-prediction-how-to-predict-your-future-connections-on-facebook)
- [ ] Read: [NetworkX](https://networkx.org/documentation/latest/)
- [ ] Read: [Graph Optimisation](https://www.datacamp.com/community/tutorials/networkx-python-graph-tutorial)
- [ ] Read: [Network Analysis](https://www.cl.cam.ac.uk/~cm542/teaching/2010/stna-pdfs/stna-lecture8.pdf)
- [ ] Read: [Pandas and Networkx](https://towardsdatascience.com/getting-started-with-graph-analysis-in-python-with-pandas-and-networkx-5e2d2f82f18e)
- [ ] Read: [Analysing Network Data](https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python)
- [x] Create networkx graph from raw data
- [ ] Use networkx graph to make prediction
- [ ] Train to obtain the weights in the networkx graph
- [ ] Follow tutorial
    - [ ] [Link Prediction Intro](https://medium.com/neo4j/link-prediction-with-neo4j-part-1-an-introduction-713aa779fd9)
    - [ ] [Link Prediction with Neo4j](https://towardsdatascience.com/link-prediction-with-neo4j-part-2-predicting-co-authors-using-scikit-learn-78b42356b44c)

In [1]:
import pandas as pd
import networkx as nx # Python package for creation, manipulation and sudy of the structure and dynamics and function of complex networks

import numpy as np
import random
from tqdm import tqdm
import re
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Functions

In [2]:
def loadTrainDataAsUndirectedGraph():
    author_list = []
    coauthor_list = []
    filename = "train.txt"
    dataRow = [line.rstrip("\n") for line in open(filename)]
    g = nx.Graph()
    for row in dataRow:
        authorIds = row.split()
        for i, author in enumerate(authorIds):
            for coauthor in authorIds[i+1:]:
                author_list.append(author)
                coauthor_list.append(coauthor)
                if g.has_edge(author, coauthor):
                    g[author][coauthor]['frequency'] += 1
                else:
                    g.add_edge(author, coauthor, frequency=1)
    return g, author_list, coauthor_list

def loadTestData():
    filename = "test-public.csv"
    return pd.read_csv(filename)

# Preprocess

In [3]:
# Load data
trainGraph, author_list, coauthor_list = loadTrainDataAsUndirectedGraph()


In [4]:
node_list = author_list + coauthor_list

# remove duplicate items from the list
node_list = list(dict.fromkeys(node_list))

# build adjacency matrix
adj_G = nx.to_numpy_matrix(trainGraph, nodelist = node_list)

In [5]:
# get unconnected node-pairs
all_unconnected_pairs = []

# traverse adjacency matrix
offset = 0
for i in tqdm(range(adj_G.shape[0])):
    for j in range(offset,adj_G.shape[1]):
        if i != j:
            try:
                if 1 < nx.shortest_path_length(trainGraph, str(i), str(j)) <= 3:
                    if adj_G[i,j] == 0:
                        all_unconnected_pairs.append([node_list[i],node_list[j]])
            except:
                continue
    offset += 1

100%|██████████| 3767/3767 [03:03<00:00, 20.57it/s] 


In [6]:
# Check graph feature: frequency (Number of times coauthoring)
print("Feature: frequency")
print('\t(10,986): ',trainGraph['10']['986']['frequency'])
print('\t(932,2861): ',trainGraph['932']['2861']['frequency'])

# Check graph feature: Shortest Path
print("Feature: shortest path")
print('\t(932,2861): ',nx.dijkstra_path(trainGraph, source='932', target='2861'))
print('\t(0,2917): ',nx.dijkstra_path(trainGraph, source='0', target='2917'))

# Check graph feature: Adamic Adar
print("Feature: adamic adar")
print('\t(932,2861): ',list(nx.adamic_adar_index(trainGraph, [('932', '2861')]))[0][2])
print('\t(0,2917): ',list(nx.adamic_adar_index(trainGraph, [('0', '2917')]))[0][2])

# Check graph feature: Common Neighbours
print("Feature: common neighbours")
print('\t(932,2861): ',len(list(nx.common_neighbors(trainGraph, '932', '2861'))))
print('\t(0,2917): ',len(list(nx.common_neighbors(trainGraph, '0', '2917'))))

# Check graph feature: Preferential Attachment
print("Feature: preferential attachment")
print('\t(932,2861): ',list(nx.preferential_attachment(trainGraph, [('932', '2861')]))[0][2])
print('\t(0,2917): ',list(nx.preferential_attachment(trainGraph, [('0', '2917')]))[0][2])

# Check graph feature: Resource Allocation
print("Feature: resource allocation")
print('\t(932,2861): ',list(nx.resource_allocation_index(trainGraph, [('932', '2861')]))[0][2])
print('\t(0,2917): ',list(nx.resource_allocation_index(trainGraph, [('0', '2917')]))[0][2])

# Check graph feature: Total Neighbours (Number of papers written)
print("Feature: total neighbours")
print('\t932: ',len(list(nx.all_neighbors(trainGraph, '932'))))
print('\t2861: ',len(list(nx.all_neighbors(trainGraph, '2861'))))
print('\t0: ',len(list(nx.all_neighbors(trainGraph, '0'))))
print('\t2917: ',len(list(nx.all_neighbors(trainGraph, '2917'))))

# Check graph feature: Jaccard Coefficient
print("Feature: jaccard coefficient")
print('\t(932,2861): ',list(nx.jaccard_coefficient(trainGraph, [('932', '2861')]))[0][2])
print('\t(0,2917): ',list(nx.jaccard_coefficient(trainGraph, [('0', '2917')]))[0][2])

# Check graph feature: Same Community
print("Feature: same community")

# Check graph feature: Triadic closure
print("Feature: triadic closure")


Feature: frequency
	(10,986):  1
	(932,2861):  35
Feature: shortest path
	(932,2861):  ['932', '2861']
	(0,2917):  ['0', '356', '3645', '2917']
Feature: adamic adar
	(932,2861):  8.329303588383727
	(0,2917):  0
Feature: common neighbours
	(932,2861):  26
	(0,2917):  0
Feature: preferential attachment
	(932,2861):  6622
	(0,2917):  56
Feature: resource allocation
	(932,2861):  1.248039538738562
	(0,2917):  0
Feature: total neighbours
	932:  77
	2861:  86
	0:  8
	2917:  7
Feature: jaccard coefficient
	(932,2861):  0.1897810218978102
	(0,2917):  0.0
Feature: same community
Feature: triadic closure


# Learn & Predict

In [25]:
# Most code below is from https://www.analyticsvidhya.com/blog/2020/01/link-prediction-how-to-predict-your-future-connections-on-facebook/

initial_node_count = len(trainGraph.nodes)

df = pd.DataFrame({'node_1': author_list, 'node_2': coauthor_list})

df_temp = df.copy()

# empty list to store removable links
omissible_links_index = []
omissible_links = []

if df_temp.shape == df.shape:
    for i in tqdm(df_temp.index.values):
      # remove a node pair and build a new graph
        try:
            author, coauthor = df_temp.iloc[i]
            G_temp = nx.from_pandas_edgelist(df_temp.drop(index = i), "node_1", "node_2", create_using=nx.Graph())

          # check there is no further spliting of graph and number of nodes is same
            if (nx.number_connected_components(G_temp) == nx.number_connected_components(trainGraph)) and (len(G_temp.nodes) == initial_node_count) and (author, coauthor) not in omissible_links:
                omissible_links_index.append(i)
                omissible_links.append((author, coauthor))
        except:
            continue
            
for i in reversed(omissible_links_index):
    df_temp = df_temp.drop(index = i)

  0%|          | 1/29620 [00:00<56:29,  8.74it/s]

(29620, 2)
(29620, 2)
(29620, 2)
(29620, 2)


 68%|██████▊   | 20091/29620 [15:50<04:13, 37.52it/s] 

Error at index 19051
Error at index 19052
Error at index 19053
Error at index 19054
Error at index 19055
Error at index 19056
Error at index 19057
Error at index 19058
Error at index 19059
Error at index 19060
Error at index 19061
Error at index 19062
Error at index 19063
Error at index 19064
Error at index 19065
Error at index 19066
Error at index 19067
Error at index 19068
Error at index 19069
Error at index 19070
Error at index 19071
Error at index 19072
Error at index 19073
Error at index 19074
Error at index 19075
Error at index 19076
Error at index 19077
Error at index 19078
Error at index 19079
Error at index 19080
Error at index 19081
Error at index 19082
Error at index 19083
Error at index 19084
Error at index 19085
Error at index 19086
Error at index 19087
Error at index 19088
Error at index 19089
Error at index 19090
Error at index 19091
Error at index 19092
Error at index 19093
Error at index 19094
Error at index 19095
Error at index 19096
Error at index 19097
Error at inde

 75%|███████▍  | 22087/29620 [15:51<01:38, 76.12it/s]

...

Error at index 25745
Error at index 25746
Error at index 25747
Error at index 25748
Error at index 25749
Error at index 25750
Error at index 25751
Error at index 25752
Error at index 25753
Error at index 25754
Error at index 25755
Error at index 25756
Error at index 25757
Error at index 25758
Error at index 25759
Error at index 25760
Error at index 25761
Error at index 25762
Error at index 25763
Error at index 25764
Error at index 25765
Error at index 25766
Error at index 25767
Error at index 25768
Error at index 25769
Error at index 25770
Error at index 25771
Error at index 25772
Error at index 25773
Error at index 25774
Error at index 25775
Error at index 25776
Error at index 25777
Error at index 25778
Error at index 25779
Error at index 25780
Error at index 25781
Error at index 25782
Error at index 25783
Error at index 25784
Error at index 25785
Error at index 25786
Error at index 25787
Error at index 25788
Error at index 25789
Error at index 25790
Error at index 25791
Error at inde

 95%|█████████▍| 28073/29620 [15:51<00:02, 601.35it/s]


Error at index 27274
Error at index 27275
Error at index 27276
Error at index 27277
Error at index 27278
Error at index 27279
Error at index 27280
Error at index 27281
Error at index 27282
Error at index 27283
Error at index 27284
Error at index 27285
Error at index 27286
Error at index 27287
Error at index 27288
Error at index 27289
Error at index 27290
Error at index 27291
Error at index 27292
Error at index 27293
Error at index 27294
Error at index 27295
Error at index 27296
Error at index 27297
Error at index 27298
Error at index 27299
Error at index 27300
Error at index 27301
Error at index 27302
Error at index 27303
Error at index 27304
Error at index 27305
Error at index 27306
Error at index 27307
Error at index 27308
Error at index 27309
Error at index 27310
Error at index 27311
Error at index 27312
Error at index 27313
Error at index 27314
Error at index 27315
Error at index 27316
Error at index 27317
Error at index 27318
Error at index 27319
Error at index 27320
Error at ind

100%|██████████| 29620/29620 [15:52<00:00, 31.11it/s] 


Error at index 29071
Error at index 29072
Error at index 29073
Error at index 29074
Error at index 29075
Error at index 29076
Error at index 29077
Error at index 29078
Error at index 29079
Error at index 29080
Error at index 29081
Error at index 29082
Error at index 29083
Error at index 29084
Error at index 29085
Error at index 29086
Error at index 29087
Error at index 29088
Error at index 29089
Error at index 29090
Error at index 29091
Error at index 29092
Error at index 29093
Error at index 29094
Error at index 29095
Error at index 29096
Error at index 29097
Error at index 29098
Error at index 29099
Error at index 29100
Error at index 29101
Error at index 29102
Error at index 29103
Error at index 29104
Error at index 29105
Error at index 29106
Error at index 29107
Error at index 29108
Error at index 29109
Error at index 29110
Error at index 29111
Error at index 29112
Error at index 29113
Error at index 29114
Error at index 29115
Error at index 29116
Error at index 29117
Error at ind

In [40]:
# create dataframe of non-existent edges (our negative samples, hence link label of 0)
author_unlinked = [i[0] for i in all_unconnected_pairs]
coauthor_unlinked = [i[1] for i in all_unconnected_pairs]

data = pd.DataFrame({'node_1':author_unlinked, 'node_2':coauthor_unlinked})
data['link'] = 0

# We do not want too big of an imbalance, so we want to down sample our negative samples, preferably with a distribution
# similar to that of our actual test (half are real, half are not, so 50%)
# Remove duplicates 
data = data.drop_duplicates()
# Down sample negative examples
data = data.sample(n=df_ghost.shape[0])


# create dataframe of removable edges (our positive samples, hence link label of 1)
df_ghost = df.loc[omissible_links_index]

# add the target variable 'link'
df_ghost['link'] = 1

# add the two dataframes together to form our training and testing set
data = data.append(df_ghost[['node_1', 'node_2', 'link']], ignore_index=True)

# drop removable edges
df_partial = df.copy()
df_partial = df_partial.drop(index=df_ghost.index.values)

In [41]:
# build graph 

G_data = nx.Graph()
for index, row in df_partial.iterrows():
    author = row["node_1"]
    coauthor = row["node_2"]
    if G_data.has_edge(author, coauthor):
        G_data[author][coauthor]['frequency'] += 1
    else:
        G_data.add_edge(author, coauthor, frequency=1)

In [44]:
from sklearn.utils import shuffle
data = shuffle(data)

In [122]:
training = data.copy()
training['Freq'] = 0
training['AdamicAdar'] = 0
training['SP'] = initial_node_count * 2
training['CN'] = 0
training['PA'] = 0
training['RA'] = 0
training['TN'] = 0
training['JC'] = 0

for index, row in training.iterrows():
    author = row["node_1"]
    coauthor = row["node_2"]
    
    if G_data.has_edge(author, coauthor):
        training.at[index,'Freq'] = G_data[author][coauthor]['frequency']
    training.at[index,'CN'] = len(list(nx.common_neighbors(G_data, author, coauthor)))
    training.at[index,'TN'] = len(list(nx.all_neighbors(G_data, author))) + len(list(nx.all_neighbors(G_data, coauthor)))
    try:
        sp = nx.dijkstra_path(G_data, source=author, target=coauthor)
        training.at[index,'SP'] = len(sp)
    except:
        continue
    
training['Source-Sink'] = list(zip(training['node_1'], training['node_2']))
training['AdamicAdar'] = [x for (_,_,x) in list(nx.adamic_adar_index(G_data, training['Source-Sink']))]
training['PA'] = [x for (_,_,x) in list(nx.preferential_attachment(G_data, training['Source-Sink']))]
training['RA'] = [x for (_,_,x) in list(nx.resource_allocation_index(G_data, training['Source-Sink']))]
training['JC'] = [x for (_,_,x) in list(nx.jaccard_coefficient(G_data, training['Source-Sink']))]
    
training = training[['node_1', 'node_2', 'Freq', 'AdamicAdar', 'SP', 'CN', 'PA', 'RA', 'TN', 'JC', 'link']]


In [123]:
xtrain, xtest, ytrain, ytest = train_test_split(training.iloc[:,:-1], training['link'], 
                                                test_size = 0.3, 
                                                random_state = 35)

In [124]:
lr = LogisticRegression(class_weight="balanced")

lr.fit(xtrain, ytrain)
predictions = lr.predict_proba(xtest)
roc_auc_score(ytest, predictions[:,1])

/Users/hochanjie/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.981618258759375

In [127]:
data

,node_1,node_2,link
5158,2444,3880,0
17713,1113,2773,1
18281,543,1228,1
7596,1402,1899,0
8810,2946,3974,0
...,...,...,...
1907,310,2656,0
4140,311,2919,0
18739,1117,3567,1
2248,378,2978,0


,Source,Sink
Id,,
1,0,2917
2,0,2956
3,1,4038
4,2,1848
5,3,513
...,...,...
1996,3865,3924
1997,3917,4025
1998,3922,3947


In [171]:
def make_features(graph, network_df):

    
    testing = network_df.copy()
    testing.Source = testing.Source.astype(str)
    testing.Sink = testing.Sink.astype(str)
    testing['Freq'] = 0
    testing['AdamicAdar'] = 0
    testing['SP'] = len(graph.nodes) * 2
    testing['CN'] = 0
    testing['PA'] = 0
    testing['RA'] = 0
    testing['TN'] = 0
    testing['JC'] = 0
    testing['Source-Sink'] = list(zip(testing['Source'], testing['Sink']))
    not_found = []

    for index, row in testing.iterrows():
        author = str(row["Source"])
        coauthor = str(row["Sink"])
        edge = row['Source-Sink']

        if graph.has_edge(author, coauthor):
            testing.at[index,'Freq'] = graph[author][coauthor]['frequency']
            
        if author in graph.nodes and coauthor in graph.nodes:
            testing.at[index,'CN'] = len(list(nx.common_neighbors(graph, author, coauthor)))
            testing.at[index,'TN'] = len(list(nx.all_neighbors(graph, author))) + len(list(nx.all_neighbors(graph, coauthor)))
            try:
                sp = nx.dijkstra_path(graph, source=author, target=coauthor)
                testing.at[index,'SP'] = len(sp)
            except:
                continue

            testing.at[index,'AdamicAdar'] = list(nx.adamic_adar_index(graph, [edge]))[0][2]
            testing.at[index,'PA'] = list(nx.preferential_attachment(graph, [edge]))[0][2]
            testing.at[index,'RA'] = list(nx.resource_allocation_index(graph, [edge]))[0][2]
            testing.at[index,'JC'] = list(nx.jaccard_coefficient(graph, [edge]))[0][2]
        else:
            if author not in graph.nodes:
                not_found.append(author)
            if coauthor not in graph.nodes:
                not_found.append(coauthor)
    testing = testing[['Source', 'Sink', 'Freq', 'AdamicAdar', 'SP', 'CN', 'PA', 'RA', 'TN', 'JC']]
    return testing, not_found

In [172]:
testing = loadTestData()
testing = testing.set_index('Id')
testing, not_found = make_features(trainGraph, testing)


In [177]:
predictions = lr.predict_proba(testing)
predictions

array([[4.38339691e-01, 5.61660309e-01],
       [6.86049052e-01, 3.13950948e-01],
       [2.14676906e-01, 7.85323094e-01],
       ...,
       [9.62528174e-01, 3.74718263e-02],
       [2.05355770e-08, 9.99999979e-01],
       [9.92028184e-01, 7.97181561e-03]])

In [185]:
final_result = pd.DataFrame(data={'Id': testing.index, 'Predicted': predictions[:,1]})
final_result

,Id,Predicted
0,1,0.561660
1,2,0.313951
2,3,0.785323
3,4,0.999960
4,5,0.477266
...,...,...
1995,1996,0.005095
1996,1997,0.000000
1997,1998,0.037472
1998,1999,1.000000


In [186]:
final_result.to_csv('linkPrediction_Cj.csv', index=False)

ModuleNotFoundError: No module named 'lightgbm'

# Evaluate